In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import time

In [13]:
df=pd.read_excel("C:/Users/xnote/Desktop/crawling/잡플래닛_기업리스트.xlsx")
df.set_index('Unnamed: 0',inplace=True)

In [132]:
import re

name0=[]
for i in range(10):
    name0.extend(df.values[:,i].tolist())
name=[]
for n in name0:
    nn=n.replace(' (매장)','')
    nn=re.compile(r'[^가-힣]+').sub(" ",nn)
    nn=nn.replace(' 주 ','')
    nn=nn.replace(' 유 ','')
    nn=nn.replace(' 재 ','')
    name.append(nn)

In [144]:
url_df = {
    '회사명' : [],
    '링크' : []
}
url_df = pd.DataFrame(url_df)
url_df

,회사명,링크


In [ ]:
import requests
import urllib.parse
from bs4 import BeautifulSoup

urls=[]
for col in name:
        baseurl = 'https://www.saramin.co.kr/zf_user/company-search?page=1&searchWord='+urllib.parse.quote_plus(col)+'&revenue=&salary=&employees=&operatingRevenue=&startingSalary=&establishment=&netRevenue='
        response = requests.get(baseurl)
        html = BeautifulSoup(response.text, 'html.parser')
        try:
            tag = html.find('div','area_info').find('a')
            salary_tag = tag.attrs['href'].replace('view','view-inner-salary')
            url_df.loc[len(url_df)] = [col,'https://www.saramin.co.kr'+salary_tag]
            print(col,'\n','https://www.saramin.co.kr'+salary_tag)
        except:
            url_df.loc[len(url_df)] = [col,'']

In [147]:
url_df['회사명']=name0

In [157]:
company_salary = {
    '회사명' : [],
    '평균연봉' : []
}
company_salary = pd.DataFrame(company_salary)
company_salary

,회사명,평균연봉


In [158]:
def crawl(name):
    data_list=[name]
    try:
        # 연봉 가져오기
        data_list.append(driver.find_element_by_css_selector('#tab_avg_salary > div > div.wrap_average > div.salaryinfo > p > em').text)
    except:
        data_list.append('')
    company_salary.loc[len(company_salary)] = data_list
    print(name)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

driver = webdriver.Chrome('C:/Program Files (x86)/Google/Chrome/Application/chromedriver')

col1=url_df['회사명'].tolist()[46:]
col2=url_df['링크'].tolist()[46:]

for name, link in zip(col1, col2):
    if link!='':
        driver.get(link)
        time.sleep(1)
        crawl(name)
        print(link)
    else:
        data_list=[name,'']
        company_salary.loc[len(company_salary)] = data_list

In [172]:
company_salary2=company_salary[company_salary['평균연봉']!='']

In [174]:
company_salary.to_excel('C:/Users/xnote/Desktop/사람인 평균연봉_빈칸O.xlsx',index=False)
company_salary2.to_excel('C:/Users/xnote/Desktop/사람인 평균연봉_빈칸X.xlsx',index=False)